In [ ]:
!pip install einops
#预装einops包，用于attention,对tensor进行操作

In [ ]:
import torch
from torch import nn
from einops import rearrange, repeat
from einops.layers.torch import Rearrange   #rearrange函数可以调整图片的维度信息

In [ ]:
# helpers 用于处理图片大小信息，统一图片大小的长宽
def pair(t):
    return t if isinstance(t, tuple) else (t, t) #判断图片信息是否是tuple类型，如果是，return自己，否则返回(t,t)
#元组的信息用()表示，且无法改变
#这一步在做什么？
#用来处理当给出的图像尺寸或块尺寸是int类型（如224）时，直接返回为同值元组（如(224, 224)）

# classes
#在做attention之前的Norm。对于embeeding的norm和对于MLP的norm
class PreNorm(nn.Module):
    def __init__(self, dim, fn):#fn应该是全联接层的简称
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):#kwargs是什么 **说明kwargs是一个字典. 扩展参数
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout))
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head * heads#总共的dim
        project_out = not (heads == 1 and dim_head == dim)#总维度不是head的维度或者不是单头注意力
        self.heads = heads
        self.scale = dim_head ** -0.5#开根号取倒数
        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        #b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim = -1)## 对tensor张量分块 x :1 197 1024   qkv 最后是一个元祖，tuple，长度是3，每个元素形状：1 197 1024
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)
        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = self.attend(dots)
        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])#先定义一个空的list
        for _ in range(depth):#对于每一个encoder，都添加attention模块和全连接层，要用layernorm进行处理
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        #这一步做了resnet的残差层
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x #最后返回的量是transformer的结果

In [ ]:
#模型主体
class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size) ## 224*224
        patch_height, patch_width = pair(patch_size)## 16 * 16
        #自查和日志
        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)
        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
        self.pool = pool
        self.to_latent = nn.Identity() #就是这个网络层的设计是用于占位的，即不干活，只是有这么一个层，放到残差网络里就是在跳过连接的地方用这个层
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes))

    def forward(self, img):
        x = self.to_patch_embedding(img) ## img 1 3 224 224  输出形状x : 1 196 1024 
        b, n, _ = x.shape ## 
        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)#class的信息加入到embedding里，动机暂时不清楚?
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]#把做完embeeding的图片信息和位置信息进行相加
        x = self.dropout(x)#这里为什么要做dropout
        x = self.transformer(x)
        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]
        x = self.to_latent(x)
        return self.mlp_head(x)

In [ ]:
#运行模型，输入超参
#默认channel的数量是3
v = ViT(
    image_size = 224,
    patch_size = 16,
    num_classes = 1000, #1000个种类
    dim = 1024, #what is this
    depth = 6,
    heads = 16,  #多头？
    mlp_dim = 2048, #fc的维度
    dropout = 0.1,
    emb_dropout = 0.1 #ebd居然也有dropout
)
#随机生成一个图片
img = torch.randn(1, 3, 224, 224)
#预测测试，并没有卵用lol
preds = v(img) # (1, 1000)